In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
)
from IPython.display import Markdown, display
import openai


openai.api_key ="" 

In [3]:
#!mkdir -p 'data/paul_graham/'
#!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt'
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()
print("Document ID:", len(documents),documents[0].doc_id)

Document ID: 1 ce5c708e-ecaf-4db8-b71a-043406e04b66


In [4]:
from llama_index import ServiceContext
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.vector_stores import VearchVectorStore

"""
集群版
"""
vector_store = VearchVectorStore( 
        path_or_url = 'http://liama-index-router.vectorbase.svc.sq01.n.jd.local',
        table_name = 'liama_index_test2',
        db_name = 'liama_index',
        flag = 1)

"""
单机版
"""
# vector_store = VearchVectorStore(         
#         path_or_url = '/data/zhx/zhx/liama_index/knowledge_base/liama_index_teststandalone',
#         # path_or_url = 'http://liama-index-router.vectorbase.svc.sq01.n.jd.local',
#         table_name = 'liama_index_teststandalone',
#         db_name = 'liama_index',
#         flag = 0)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
service_context = ServiceContext.from_defaults(embed_model=embed_model)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
display(Markdown(f"<b>{response}</b>"))


<b>The author mentions that they spent some time writing essays in their youth, but除此之外, there is no information provided about their growing up.</b>

In [6]:

query_engine = index.as_query_engine()
response = query_engine.query(
    "What did the author do after his time at Y Combinator?"
)
display(Markdown(f"<b>{response}</b>"))

<b>The author after his time at Y Combinator worked on a startup called HN and then transitioned to working on Y Combinator. He also wrote all of Y Combinator's internal software in Arc while he continued to work on HN. Ultimately, the author realized that Y Combinator was taking up too much of his attention and decided to hand it over to someone else.</b>